In [84]:
import os
directory = os.getcwd()
res_folder = '\\trn'
os.chdir(directory+res_folder)

In [85]:
test = os.listdir()
avl_labels = [] # avaliable labels in the 
for i in test:
    if '_temp_flow' in i:
        prefix = i[:-14]
    elif '_control_signal' in i:
        prefix = i[:-19]
    elif '_energy' in i:
        prefix = i[:-11]
    else:
        continue
    avl_labels.append(prefix)
        
avl_labels

['0',
 '0',
 '0',
 '10',
 '10',
 '10',
 '11',
 '11',
 '11',
 '12',
 '12',
 '12',
 '13',
 '13',
 '13',
 '14',
 '14',
 '14',
 '15',
 '15',
 '15',
 '16',
 '16',
 '16',
 '17',
 '17',
 '17',
 '18',
 '18',
 '18',
 '19',
 '19',
 '19',
 '1',
 '1',
 '1',
 '20',
 '20',
 '20',
 '21',
 '21',
 '21',
 '22',
 '22',
 '22',
 '23',
 '23',
 '23',
 '24',
 '24',
 '24',
 '25',
 '25',
 '25',
 '26',
 '26',
 '26',
 '27',
 '27',
 '27',
 '28',
 '28',
 '28',
 '29',
 '29',
 '29',
 '2',
 '2',
 '2',
 '30',
 '30',
 '30',
 '31',
 '31',
 '31',
 '32',
 '32',
 '32',
 '33',
 '33',
 '33',
 '34',
 '34',
 '34',
 '35',
 '35',
 '35',
 '36',
 '36',
 '36',
 '37',
 '37',
 '37',
 '38',
 '38',
 '38',
 '39',
 '39',
 '39',
 '3',
 '3',
 '3',
 '40',
 '40',
 '40',
 '41',
 '41',
 '41',
 '42',
 '42',
 '42',
 '43',
 '43',
 '43',
 '44',
 '44',
 '44',
 '45',
 '45',
 '45',
 '46',
 '46',
 '46',
 '47',
 '47',
 '47',
 '48',
 '48',
 '48',
 '49',
 '49',
 '49',
 '4',
 '4',
 '4',
 '50',
 '50',
 '50',
 '51',
 '51',
 '51',
 '52',
 '52',
 '52',
 '53',


In [86]:
import numpy as np
avl_labels = np.array(avl_labels)
x = np.unique(avl_labels)
x

array(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58',
       '59', '6', '60', '61', '62', '63', '64', '65', '7', '8', '9',
       'PVT_V0_1_A16', 'PVT_V0_1_A20', 'PVT_V0_1_A8', 'PVT_V0_2_A16',
       'PVT_V0_2_A20', 'PVT_V0_2_A8', 'PVT_V0_3_A16', 'PVT_V0_3_A20',
       'PVT_V0_3_A8', 'PVT_V0_4_A16', 'PVT_V0_4_A20', 'PVT_V0_4_A8',
       'cp_PV_V0_1_A16', 'cp_PV_V0_1_A20', 'cp_PV_V0_1_A8',
       'cp_PV_V0_2_A16', 'cp_PV_V0_2_A20', 'cp_PV_V0_2_A8',
       'cp_PV_V0_3_A16', 'cp_PV_V0_3_A20', 'cp_PV_V0_3_A8',
       'cp_PV_V0_4_A16', 'cp_PV_V0_4_A20', 'cp_PV_V0_4_A8', 'cp_V0_1_A16',
       'cp_V0_1_A20', 'cp_V0_1_A8', 'cp_V0_2_A16', 'cp_V0_2_A20',
       'cp_V0_2_A8', 'cp_V0_3_A16', 'cp_V0

In [87]:
len(avl_labels)

306

In [88]:
list_sample = pd.DataFrame(columns = ['flow_rate','volume','area'])
for i in np.arange(30):
    parameters = pd.read_csv(str(i)+'.txt', delimiter='\t', header=None)
    parameters = parameters.transpose()
    parameters.columns = parameters.iloc[0]
    parameters=parameters.drop(parameters.index[0])
    list_sample.loc[i]=parameters.iloc[0]
    


In [89]:
list_sample

,flow_rate,volume,area
0,200.0,0.4,8.0
1,200.0,0.2,8.0
2,200.0,0.2,20.0
3,100.0,0.2,20.0
4,100.0,0.1,8.0
5,100.0,0.1,20.0
6,100.0,0.3,20.0
7,200.0,0.3,20.0
8,100.0,0.3,4.0
9,100.0,0.1,4.0


In [10]:
import pandas as pd
os.chdir('C:/Users/20181270/OneDrive - TU Eindhoven/PhD/TRNSYS/Publication1/pub_1/src/res')

df1 = pd.read_csv('morris_st_sample.csv')
df2 = pd.read_csv('morris_pvt_sample.csv')
dfmorris = pd.concat([df1,df2])
df = dfmorris.drop_duplicates()
df.index = np.arange(len(df))

In [11]:
print(len(df1))
print(len(df2))
print(len(df))

128
128
86


In [36]:
sobol_labels = []
for i in range(len(df)):
    sobol_labels.append(dfsobol['design_case'][i]+'_V'+str(dfsobol['volume'][i]).replace('.','_')+'_A'+str(dfsobol['coll_area'][i]))
sobol_labels = np.array(sobol_labels)
len(sobol_labels)

86

In [42]:
result = np.concatenate((labels, sobol_labels))

In [43]:
# missing samples from simulation results
set(sobol_labels) - set(labels2)

{'ST_V0_1_A16',
 'ST_V0_1_A20',
 'ST_V0_1_A8',
 'ST_V0_2_A16',
 'ST_V0_2_A20',
 'ST_V0_2_A8',
 'ST_V0_3_A16',
 'ST_V0_3_A20',
 'ST_V0_3_A8',
 'ST_V0_4_A16',
 'ST_V0_4_A20',
 'ST_V0_4_A8'}

In [33]:
sobol_label2 = dfsobol.drop_duplicates()
len(sobol_label2)

86